# 배열끼리 값 더하기
***

두개의 numpy array를 서로 합산

In [1]:
import numpy as np

a_arr = np.random.randint(10, size=6)
b_arr = np.random.randint(10, size=6)
sum_arr = a_arr + b_arr

print("a_arr =")
print(a_arr)
print("\nb_arr =")
print(b_arr)
print("\nsum_arr =")
print(sum_arr)

a_arr =
[6 0 7 8 5 4]

b_arr =
[4 1 4 1 4 3]

sum_arr =
[10  1 11  9  9  7]


### Block 1개, Thread 6개 사용

In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda import driver, compiler

In [3]:
# Kernel code
kernel_code = """
__global__ void add(int* in_arr1, int* in_arr2, int* out_arr)
{
  int idx = threadIdx.x;
  out_arr[idx] = in_arr1[idx] + in_arr2[idx];
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# Get kernel function
add_func = mod.get_function("add")

# Run
result_arr = np.zeros_like(a_arr)
add_func(cuda.In(a_arr), cuda.In(b_arr), cuda.Out(result_arr), block=(6, 1, 1))

# 출력
print("\nGPU로 계산 =")
print(result_arr)
print("\nCPU로 계산 =")
print(sum_arr)


GPU로 계산 =
[10  1 11  9  9  7]

CPU로 계산 =
[10  1 11  9  9  7]


### Block 2개, Block당 Thread 3개 사용 (총 6개의 Thread)

In [4]:
# Kernel code
kernel_code = """
__global__ void add(int* in_arr1, int* in_arr2, int* out_arr)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  out_arr[idx] = in_arr1[idx] + in_arr2[idx];
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# Get kernel function
add_func = mod.get_function("add")

# Run
result_arr = np.zeros_like(a_arr)
add_func(cuda.In(a_arr), cuda.In(b_arr), cuda.Out(result_arr), block=(3, 1, 1), grid=(2, 1))

# 출력
print("\nGPU로 계산 =")
print(result_arr)
print("\nCPU로 계산 =")
print(sum_arr)


GPU로 계산 =
[10  1 11  9  9  7]

CPU로 계산 =
[10  1 11  9  9  7]


#### 한번 컴파일 해놓으면 재사용 가능

In [5]:
print("a_arr =")
print(a_arr)

print("\n누적하여 더하기")
result_arr = np.zeros_like(a_arr)
print(result_arr)

add_func(cuda.In(a_arr), cuda.In(result_arr), cuda.Out(result_arr), block=(3, 1, 1), grid=(2, 1))
print(result_arr)

add_func(cuda.In(a_arr), cuda.In(result_arr), cuda.Out(result_arr), block=(3, 1, 1), grid=(2, 1))
print(result_arr)

add_func(cuda.In(a_arr), cuda.In(result_arr), cuda.Out(result_arr), block=(3, 1, 1), grid=(2, 1))
print(result_arr)

a_arr =
[6 0 7 8 5 4]

누적하여 더하기
[0 0 0 0 0 0]
[6 0 7 8 5 4]
[12  0 14 16 10  8]
[18  0 21 24 15 12]


### Thread의 갯수보다 더 큰 Size의 배열 처리
* numpy array의 size는 10개
* Block 2개, Block당 Thread 3개 사용 (총 6개의 Thread)

In [6]:
a_arr = np.random.randint(10, size=10)
b_arr = np.random.randint(10, size=10)
sum_arr = a_arr + b_arr

print("a_arr =")
print(a_arr)
print("\nb_arr =")
print(b_arr)
print("\nsum_arr =")
print(sum_arr)

a_arr =
[7 1 8 0 1 1 1 5 0 0]

b_arr =
[8 6 9 2 4 2 7 9 7 3]

sum_arr =
[15  7 17  2  5  3  8 14  7  3]


In [7]:
# Kernel code
kernel_code = """
__device__ __constant__ int out_arr_size;

__global__ void add(int* in_arr1, int* in_arr2, int* out_arr)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  
  while ( idx < out_arr_size )
  {
      out_arr[idx] = in_arr1[idx] + in_arr2[idx];
      idx += blockDim.x * gridDim.x;
  }
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# 결과를 받을 array
result_arr = np.zeros_like(a_arr)

# 상수 설정
ret_arr_size = np.array([result_arr.size], dtype=np.int)
device_arr_size = mod.get_global("out_arr_size")[0]
cuda.memcpy_htod(device_arr_size, ret_arr_size[0])

# Get kernel function
add_func = mod.get_function("add")

# Run
add_func(cuda.In(a_arr), cuda.In(b_arr), cuda.Out(result_arr), block=(3, 1, 1), grid=(2, 1))

# 출력
print("\nGPU로 계산 =")
print(result_arr)
print("\nCPU로 계산 =")
print(sum_arr)


GPU로 계산 =
[15  7 17  2  5  3  8 14  7  3]

CPU로 계산 =
[15  7 17  2  5  3  8 14  7  3]
